# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

* Please write code *only* in the bodies of the two functions, that is, following the TODO comments.
* Be careful not to use varibles defined outside of the functions.
* Breaking the two above rules may lead to 0 grades.

## Movie distribution

Recall Problem 1 from hw3t. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.nodes[state]['demand'] = 1
G.nodes['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edges[s1, s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    """Computes a flow with demands over the given graph.
    
    Args:
        graph: A directed graph with nodes annotated with 'demand' properties and edges annotated with 'capacity' 
            properties.
        
    Returns:
        A dict of dicts containing the flow on each edge. For instance, flow[s1][s2] should provide the flow along
        edge (s1, s2).
        
    Raises:
        NetworkXUnfeasible: An error is thrown if there is no flow satisfying the demands.
    """
    ## check the necessary condition for a feasible flow to exist
    sum_demands = 0
    for node in graph.nodes():
        sum_demands += graph.nodes[node]['demand']
    
    if sum_demands != 0:
        raise nx.NetworkXUnfeasible('An error is thrown if there is no flow satisfying the demands.')
    
    
    ## transformation
    G = graph.copy()
    
    # add super-source node
    G.add_node('super_source_node')
    
    # add super-sink node
    G.add_node('super_sink_node')
    
    for node in graph.nodes():
        
        # if node is a sink node
        if graph.nodes[node]['demand'] > 0:
            # connect sink node to super-sink node
            G.add_edge(node, 'super_sink_node')
            # set capacity on the new edge
            G.edges[node, 'super_sink_node']['capacity'] = graph.nodes[node]['demand']

        # if node is a source node
        elif graph.nodes[node]['demand'] < 0:
            # connect super-source node to source node
            G.add_edge('super_source_node', node)
            # set capacity on the new edge
            G.edges['super_source_node', node]['capacity'] = -graph.nodes[node]['demand']
                
                
    ## run max-flow (Edmonds Karp's) on tranformed network to get max-flow
    flow_size, flow = nx.maximum_flow(flowG=G, _s='super_source_node', _t='super_sink_node', flow_func=nx.algorithms.flow.edmonds_karp)
    
    
    ## check if feasible flow exists 
    sink_demands_sum = 0
    for node in graph.nodes():
        if graph.nodes[node]['demand'] > 0:
            sink_demands_sum += graph.nodes[node]['demand']
            
    if flow_size != sink_demands_sum:
        raise nx.NetworkXUnfeasible('An error is thrown if there is no flow satisfying the demands.')
        
    
    ## return flow in original graph
    t = ['super_source_node', 'super_sink_node']
    return {key:{key_:flow[key][key_] for key_ in flow[key] if key_ not in t} for key in flow if key not in t}


To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    """Computes the total flow into each node according to the given flow dict.
    
    Args:
        flow: the flow dict recording flow between nodes.
        
    Returns:
        A dict of the net flow into each node.
    """
    flow_in = {node:0 for node in flow}
    flow_out = {node:0 for node in flow}
    
    for u in flow:
        for v in flow[u]:
            flow_out[u] += flow[u][v]
            flow_in[v] += flow[u][v]
            
    return {node:flow_in[node] - flow_out[node] for node in flow}

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
# flow = flow_with_demands(G)
# div = divergence(flow)
# print ("Flow satisfies all demands:", all(div[n] == G.nodes[n]['demand'] for n in G.nodes()))